# 時系列データを用いた機械学習
時系列データに対する機械学習手法（Lasso, Ridge, サポートベクターマシン, ランダムフォレスト、ブースティング木、XGBoost、LightGBM）の使用例。

参考：https://code-examples.net/ja/docs/scikit_learn/modules/cross_validation

参考：https://qiita.com/nazoking@github/items/1c5106a844084cd8539a

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

from sklearn import linear_model
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
import lightgbm as lgb

### インプットデータファイル。
ここでは、単なる動作用のサンプルとして適当に作ったdata.csvを使用。

サイズは1000×4の行列。説明変数 X1, X2, X3, 被説明変数 Yとする。時刻がt = 1,..., 1000。

このdata.csvファイルは Y(t) = 3 * X1(t) + X2(t) - X3(t) + (ノイズ) (t) という式で人工的に作成している。

任意のインプットデータファイルを使いたければ、この部分を変更する必要がある。

In [2]:
df = pd.read_csv('data.csv')

X = df[["X1", "X2", "X3"]].values
y = df[["Y"]].values
y = np.reshape(y,(-1)) # ←yを1次元配列に変換するために必要。

df.head() # 最初の5つだけ表示。

,X1,X2,X3,Y
0,0.941956,0.125264,-0.732235,3.620466
1,0.572490,-0.463934,-0.858543,2.156731
2,0.855720,0.467162,-0.565409,3.620363
3,0.050989,0.271136,-0.901203,1.347712
4,0.450075,0.921557,-0.508214,2.784960


### trainデータとtestデータに分割。
今回はデータ全体の90%をtrainデータ、10%をtestデータとする。

この90%のtrainデータを用いてパラメータとハイパーパラメータを決める。つまり、このtrainデータを後で更にtrain/testに分解し、クロスバリデーションを行う。

10%のtestデータは最終的な精度評価に用いる。

In [3]:
n_train = int(len(X) * 0.9)  # 今回は、1000個中の90%で900個がtrainデータ。
X_train = X[:n_train]        # 1～900の900個
X_test  = X[n_train:]        # 901～1000の100個
y_train = y[:n_train]        # 1～900の900個
y_test  = y[n_train:]        # 901～1000の100個

### クロスバリデーションにおける分割方法の指定。
時系列データでのクロスバリデーションはやや特殊。未来のデータを予測に使ってしまわないように注意して分割する必要がある。

In [4]:
# 最も愚直に、手作業で指定する方法がこちら。

# ハイパーパラメータを仮置きし、400個の過去データからパラメータを決め、100個の将来データで精度を評価、
# という作業をウィンドウをずらしながら5回繰り返すものとする。

tscv  = [(np.arange(  0, 400), np.arange(400, 500)),
         (np.arange(100, 500), np.arange(500, 600)),
         (np.arange(200, 600), np.arange(600, 700)),
         (np.arange(300, 700), np.arange(700, 800)),
         (np.arange(400, 800), np.arange(800, 900))] # こうして作ったtscvオブジェクトを、後のGridSearchCV関数の引数として使用する。

TimeSeriesSplit関数を使うこともできるが、分け方がイマイチか。

tscv = TimeSeriesSplit(n_splits=5)

これだとデータ900個を6等分し、

1回目：第1グループ150個の過去データからパラメータを決め、第2グループ150個の将来データで精度を評価、

2回目：第1+第2グループ300個の過去データからパラメータを決め、第3グループ150個の将来データで精度を評価、

3回目：第1+第2+第3グループ450個の過去データからパラメータを決め、第4グループ150個の将来データで精度を評価、

4回目：第1+第2+第3+第4グループ600個の過去データからパラメータを決め、第5グループ150個の将来データで精度を評価、

5回目：第1+第2+第3+第4+第5グループ750個の過去データからパラメータを決め、第6グループ150個の将来データで精度を評価、

というクロスバリデーションを行ってしまうので、1回目のパラメータ推計で用いられるデータの個数がどうしても少なくなる。

# 各機械学習手法を実行
Lasso, Ridge, サポートベクターマシン, ランダムフォレスト、ブースティング木、XGBoost、LightGBM

ただしXGBoostはpipによる簡単なインストールができないので、使えるようにするためには下記書籍に記載されているような面倒な作業が必要。
https://www.amazon.co.jp/dp/4254275838

#### 基本的にはGridSearchCVでクロスバリデーションを実行。候補の中から最適なハイパーパラメータを選択。
参考：https://qiita.com/Lewuathe/items/09d07d3ff366e0dd6b24
#### ただしLasso、Ridgeなど、 手法によっては専用のCV機能がついているものもあるので、それを使ってもよい。
参考：http://scikit-learn.org/stable/modules/grid_search.html#grid-search

# 1. Lasso
http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html#sklearn.linear_model.LassoCV

In [5]:
#### クロスバリデーション実行
lasso = linear_model.LassoCV(cv=tscv)   # ここで分割方法を指定できる。今回はcv=tscv。
lasso.fit(X_train, y_train)

#### CVの諸設定を出力
print(lasso)

#### クロスバリデーションで得たハイパーパラメータ
print(' ')
print('Lasso alpha is {}'.format(lasso.alpha_))

#### クロスバリデーションで得たパラメータおよびハイパーパラメータをもとに、予測実行
lasso.pred = lasso.predict(X_test)

#### 精度評価
print(' ')
print('R2 is {}'.format(r2_score(y_test, lasso.pred)) )
print('RMSR is {}'.format(np.sqrt(mean_squared_error(y_test, lasso.pred))) )

LassoCV(alphas=None, copy_X=True,
    cv=[(array([  0,   1, ..., 398, 399]), array([400, 401, ..., 498, 499])), (array([100, 101, ..., 498, 499]), array([500, 501, ..., 598, 599])), (array([200, 201, ..., 598, 599]), array([600, 601, ..., 698, 699])), (array([300, 301, ..., 698, 699]), array([700, 701, ..., 798, 799])), (array([400, 401, ..., 798, 799]), array([800, 801, ..., 898, 899]))],
    eps=0.001, fit_intercept=True, max_iter=1000, n_alphas=100, n_jobs=1,
    normalize=False, positive=False, precompute='auto', random_state=None,
    selection='cyclic', tol=0.0001, verbose=False)
 
Lasso alpha is 0.00033084068863525006
 
R2 is 0.99636148274248
RMSR is 0.06755423785949705


# 2. Ridge
http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html#sklearn.linear_model.LassoCV

In [6]:
#### クロスバリデーション実行
ridge = linear_model.RidgeCV(alphas=(0.001, 0.01, 0.1, 1.0, 10.0), cv=tscv) # Ridgeのハイパーパラメータ候補を手入力。
ridge.fit(X_train, y_train)

#### CVの諸設定を出力
print(ridge)

#### クロスバリデーションで得たハイパーパラメータ
print(' ')
print('Ridge alpha is {}'.format(ridge.alpha_))

#### クロスバリデーションで得たパラメータおよびハイパーパラメータをもとに、予測実行
ridge.pred = ridge.predict(X_test)

#### 精度評価
print(' ')
print('R2 is {}'.format(r2_score(y_test, ridge.pred)) )
print('RMSR is {}'.format(np.sqrt(mean_squared_error(y_test, ridge.pred))) )

RidgeCV(alphas=(0.001, 0.01, 0.1, 1.0, 10.0),
    cv=[(array([  0,   1, ..., 398, 399]), array([400, 401, ..., 498, 499])), (array([100, 101, ..., 498, 499]), array([500, 501, ..., 598, 599])), (array([200, 201, ..., 598, 599]), array([600, 601, ..., 698, 699])), (array([300, 301, ..., 698, 699]), array([700, 701, ..., 798, 799])), (array([400, 401, ..., 798, 799]), array([800, 801, ..., 898, 899]))],
    fit_intercept=True, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)
 
Ridge alpha is 0.01
 
R2 is 0.9963678294750382
RMSR is 0.06749529409445437


# 3. サポートベクターマシン

In [7]:
svr = svm.SVR(kernel='linear')

#### ハイパーパラメータ候補
param_grid = [
    {'C': [0.01, 0.1, 0.5], 'kernel': ['linear']},
    {'C': [0.01, 0.1, 0.5], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
]

#### クロスバリデーション実行
reg = GridSearchCV(estimator=svr,
                   param_grid=param_grid,
                   cv=tscv,
                   scoring='neg_mean_squared_error',
                   n_jobs=-1,
                   return_train_score=True)
reg.fit(X_train, y_train) 

#### クロスバリデーションで得たハイパーパラメータ
print(reg.best_estimator_)

#### クロスバリデーションで得たパラメータおよびハイパーパラメータをもとに、予測実行
predictor=reg.best_estimator_
svm_pred = predictor.predict(X_test)

#### 精度評価
print(' ')
print('R2 is {}'.format(r2_score(y_test, svm_pred)) )
print('RMSR is {}'.format(np.sqrt(mean_squared_error(y_test, svm_pred))) )

SVR(C=0.5, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
 
R2 is 0.996348007775005
RMSR is 0.06767921320692082


#### サポートベクターマシンのCVで上位スコアを出したハイパーパラメータを表示
参考：http://www.procrasist.com/entry/10-cross-validation

In [8]:
df = pd.DataFrame(reg.cv_results_)
df.sort_values(by=["rank_test_score"])[["params","mean_test_score","std_test_score","mean_fit_time"]].head()

,params,mean_test_score,std_test_score,mean_fit_time
2,"{'C': 0.5, 'kernel': 'linear'}",-0.004771,0.000474,0.009418
1,"{'C': 0.1, 'kernel': 'linear'}",-0.008620,0.000878,0.005161
0,"{'C': 0.01, 'kernel': 'linear'}",-0.481637,0.031379,0.006811
7,"{'C': 0.5, 'gamma': 0.001, 'kernel': 'rbf'}",-1.071478,0.050322,0.010105
5,"{'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}",-1.154722,0.055038,0.010713


# 4. ランダムフォレスト
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor

https://data-science.gr.jp/implementation/iml_sklearn_random_forest.html

In [9]:
rf = RandomForestRegressor(random_state=0)

#### ハイパーパラメータ候補
param_grid = {
    "n_estimators" :[i for i in range(10,200,10)],
    "max_depth"    :[i for i in range(1,10,1)]
    }

#### クロスバリデーション実行
reg = GridSearchCV(estimator=rf,
                   param_grid=param_grid,
                   cv=tscv,
                   scoring='neg_mean_squared_error',
                   n_jobs=-1,
                   return_train_score=True)
reg.fit(X_train, y_train)

#### クロスバリデーションで得たハイパーパラメータ
print(reg.best_estimator_)
    
#### クロスバリデーションで得たパラメータおよびハイパーパラメータをもとに、予測実行
predictor=reg.best_estimator_
rf.pred = predictor.predict(X_test)

#### 精度評価
print(' ')
print('R2 is {}'.format(r2_score(y_test, rf.pred)) )
print('RMSR is {}'.format(np.sqrt(mean_squared_error(y_test, rf.pred))) )

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=9,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)
 
R2 is 0.9905796842272377
RMSR is 0.10869838949166469


#### ランダムフォレストのCVで上位スコアを出したハイパーパラメータを表示

In [10]:
df = pd.DataFrame(reg.cv_results_)
df.sort_values(by=["rank_test_score"])[["params","mean_test_score","std_test_score","mean_fit_time"]].head()

,params,mean_test_score,std_test_score,mean_fit_time
161,"{'max_depth': 9, 'n_estimators': 100}",-0.020804,0.003736,0.440557
162,"{'max_depth': 9, 'n_estimators': 110}",-0.020805,0.003789,0.498207
164,"{'max_depth': 9, 'n_estimators': 130}",-0.020817,0.003836,0.582510
160,"{'max_depth': 9, 'n_estimators': 90}",-0.020834,0.003710,0.387803
163,"{'max_depth': 9, 'n_estimators': 120}",-0.020881,0.003792,0.538078


# 5. ブースティング木
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html#sklearn.ensemble.GradientBoostingRegressor

https://qiita.com/nazoking@github/items/51a46256ecda598b60dd

In [11]:
gb = GradientBoostingRegressor(random_state=0)

#### ハイパーパラメータ候補
param_grid = {
    "n_estimators" :[i for i in range(10,200,10)],
    "max_depth"    :[i for i in range(1,10,1)]
    }

#### クロスバリデーション実行
reg = GridSearchCV(estimator=gb,
                   param_grid=param_grid,
                   cv=tscv,
                   scoring='neg_mean_squared_error',
                   n_jobs=-1,
                   return_train_score=True)
reg.fit(X_train, y_train)

#### クロスバリデーションで得たハイパーパラメータ
print(reg.best_estimator_)
    
#### クロスバリデーションで得たパラメータおよびハイパーパラメータをもとに、予測実行
predictor=reg.best_estimator_
gb.pred = predictor.predict(X_test)

#### 精度評価
print(' ')
print('R2 is {}'.format(r2_score(y_test, gb.pred)) )
print('RMSR is {}'.format(np.sqrt(mean_squared_error(y_test, gb.pred))) )

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=190, presort='auto', random_state=0,
             subsample=1.0, verbose=0, warm_start=False)
 
R2 is 0.9923879911605268
RMSR is 0.09771022674078265


#### ブースティング木のCVで上位スコアを出したハイパーパラメータを表示

In [12]:
df = pd.DataFrame(reg.cv_results_)
df.sort_values(by=["rank_test_score"])[["params","mean_test_score","std_test_score","mean_fit_time"]].head()

,params,mean_test_score,std_test_score,mean_fit_time
37,"{'max_depth': 2, 'n_estimators': 190}",-0.013474,0.002953,0.098885
56,"{'max_depth': 3, 'n_estimators': 190}",-0.013552,0.002805,0.121816
55,"{'max_depth': 3, 'n_estimators': 180}",-0.013629,0.002846,0.107788
36,"{'max_depth': 2, 'n_estimators': 180}",-0.013644,0.002955,0.088464
54,"{'max_depth': 3, 'n_estimators': 170}",-0.013670,0.002869,0.102189


# 6. ニューラルネット（多層パーセプトロン）
公式：http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor

シンプルな計算例：https://qiita.com/mix_dvd/items/c37ef06b1ba58bf6d578

丁寧なパラメータ解説：https://data-science.gr.jp/implementation/iml_sklearn_mlp.html

丁寧なパラメータ解説：https://spjai.com/neural-network-parameter/

In [13]:
nnet = MLPRegressor(random_state=0, max_iter=10000) # early_stopping=Trueはすべきか 

#### ハイパーパラメータ候補
param_grid = {
    "hidden_layer_sizes" : [(10), (10, 5), (10, 10, 5), (10, 10, 10, 5)],
    "batch_size" : [20,50,100,200]
    }

#### クロスバリデーション実行
reg = GridSearchCV(estimator=nnet,
                   param_grid=param_grid,
                   cv=tscv,
                   scoring='neg_mean_squared_error',
                   n_jobs=-1,
                   return_train_score=True)
reg.fit(X_train, y_train)

#### クロスバリデーションで得たハイパーパラメータ
print(reg.best_estimator_)
    
#### クロスバリデーションで得たパラメータおよびハイパーパラメータをもとに、予測実行
predictor=reg.best_estimator_
nnet.pred = predictor.predict(X_test)

#### 精度評価
print(' ')
print('R2 is {}'.format(r2_score(y_test, nnet.pred)) )
print('RMSR is {}'.format(np.sqrt(mean_squared_error(y_test, nnet.pred))) )

MLPRegressor(activation='relu', alpha=0.0001, batch_size=20, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 5), learning_rate='constant',
       learning_rate_init=0.001, max_iter=10000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=0, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)
 
R2 is 0.996188648658169
RMSR is 0.06914007904357049


#### ニューラルネットのCVで上位スコアを出したハイパーパラメータを表示

In [14]:
df = pd.DataFrame(reg.cv_results_)
df.sort_values(by=["rank_test_score"])[["params","mean_test_score","std_test_score","mean_fit_time"]].head()

,params,mean_test_score,std_test_score,mean_fit_time
1,"{'batch_size': 20, 'hidden_layer_sizes': (10, 5)}",-0.005527,0.001030,0.933266
5,"{'batch_size': 50, 'hidden_layer_sizes': (10, 5)}",-0.006580,0.001154,0.923160
3,"{'batch_size': 20, 'hidden_layer_sizes': (10, ...",-0.007713,0.001430,1.964085
9,"{'batch_size': 100, 'hidden_layer_sizes': (10,...",-0.008976,0.001711,0.883088
0,"{'batch_size': 20, 'hidden_layer_sizes': 10}",-0.011585,0.001288,0.870888


# 7. XGBoost
http://tekenuko.hatenablog.com/entry/2016/09/22/220814

https://qiita.com/onlyzs/items/46305d3ca41f0a764b55

In [15]:
xgb_model = xgb.XGBRegressor(random_state=0)

#### ハイパーパラメータ候補
param_grid = {
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.05, 0.1],
    'subsample': [0.8, 0.85, 0.9, 0.95]
    }

#### クロスバリデーション実行
reg = GridSearchCV(estimator=xgb_model,
                   param_grid=param_grid,
                   cv=tscv,
                   scoring='neg_mean_squared_error', 
                   n_jobs=-1,
                   return_train_score=True)
reg.fit(X_train, y_train)

#### クロスバリデーションで得たハイパーパラメータ
print(reg.best_estimator_)

#### クロスバリデーションで得たパラメータおよびハイパーパラメータをもとに、予測実行
predictor = reg.best_estimator_
xgb_pred = predictor.predict(X_test)

#### 精度評価
print(' ')
print('R2 is {}'.format(r2_score(y_test, xgb_pred)) )
print('RMSR is {}'.format(np.sqrt(mean_squared_error(y_test, xgb_pred))) )

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)
 
R2 is 0.9908555749513108
RMSR is 0.10709484865192571


#### XGBoostのCVで上位スコアを出したハイパーパラメータを表示

In [16]:
pd.set_option("display.max_colwidth", 150)
df = pd.DataFrame(reg.cv_results_)
df.sort_values(by=["rank_test_score"])[["params","mean_test_score","std_test_score"]].head()

,params,mean_test_score,std_test_score
24,"{'learning_rate': 0.1, 'max_depth': 7, 'subsample': 0.8}",-0.013545,0.001294
28,"{'learning_rate': 0.1, 'max_depth': 10, 'subsample': 0.8}",-0.013549,0.001518
16,"{'learning_rate': 0.1, 'max_depth': 3, 'subsample': 0.8}",-0.013774,0.003820
20,"{'learning_rate': 0.1, 'max_depth': 5, 'subsample': 0.8}",-0.013783,0.001721
21,"{'learning_rate': 0.1, 'max_depth': 5, 'subsample': 0.85}",-0.014027,0.002243


# 8. LightGBM
https://blog.amedama.jp/entry/2018/05/01/081842

https://stackoverrun.com/ja/q/12370570

http://longtweets.hatenablog.com/?page=1520156276

https://qiita.com/TomokIshii/items/3729c1b9c658cc48b5cb

In [17]:
lgb_model = lgb.LGBMRegressor(random_state=0)

#### ハイパーパラメータ候補
param_grid = {
    'objective' : ['regression'],
    'max_depth': [-1, 2, 3, 5, 7, 10],
    'num_leaves' : [2, 4, 8, 31, 32, 128, 1024],
    'learning_rate': [0.05, 0.1]
    }

#### クロスバリデーション実行
reg = GridSearchCV(estimator=lgb_model,
                   param_grid=param_grid,
                   cv=tscv,
                   scoring='neg_mean_squared_error', # 'mean_squared_error' と書いても同じ。
                   n_jobs=-1,
                   return_train_score=True)
reg.fit(X_train, y_train)

#### クロスバリデーションで得たハイパーパラメータ
print(reg.best_estimator_)
    
#### クロスバリデーションで得たパラメータおよびハイパーパラメータをもとに、予測実行
predictor = reg.best_estimator_
lgb_pred = predictor.predict(X_test)

#### 精度評価
print(' ')
print('R2 is {}'.format(r2_score(y_test, lgb_pred)) )
print('RMSR is {}'.format(np.sqrt(mean_squared_error(y_test, lgb_pred))) )

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
       n_jobs=-1, num_leaves=8, objective='regression', random_state=0,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=1)
 
R2 is 0.991058878918213
RMSR is 0.1058976608574778


#### LightGBMのCVで上位スコアを出したハイパーパラメータを表示

In [18]:
df = pd.DataFrame(reg.cv_results_)
df.sort_values(by=["rank_test_score"])[["params","mean_test_score","std_test_score"]].head()

,params,mean_test_score,std_test_score
44,"{'learning_rate': 0.1, 'max_depth': -1, 'num_leaves': 8, 'objective': 'regression'}",-0.014666,0.002211
72,"{'learning_rate': 0.1, 'max_depth': 7, 'num_leaves': 8, 'objective': 'regression'}",-0.014666,0.002211
79,"{'learning_rate': 0.1, 'max_depth': 10, 'num_leaves': 8, 'objective': 'regression'}",-0.014666,0.002211
45,"{'learning_rate': 0.1, 'max_depth': -1, 'num_leaves': 31, 'objective': 'regression'}",-0.014767,0.002814
46,"{'learning_rate': 0.1, 'max_depth': -1, 'num_leaves': 32, 'objective': 'regression'}",-0.014767,0.002814
